# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [3]:
def exp_return(data, P):
    exp_ann = 0
    for x in range(len(data["Price in 1 Year"])):
        exp_ann += ((data["Price in 1 Year"][x] / P) - 1) * data["Probability"][x]
    # exp_return(data, 40) should output 0.07500000000000002
    return exp_ann

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [4]:
def volatility(data, P):
    exp_ann_2 = 0
    for x in range(len(data["Price in 1 Year"])):
        exp_ann_2 += (((data["Price in 1 Year"][x] / P) - 1) ** 2) * data["Probability"][x]
    exp_ann_2 -= exp_return(data, P) ** 2
    # volatility(data,40) should output 0.21301408404140795
    return exp_ann_2 ** (1/2)

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [5]:
def interval(data, P):
    interval = [0, 0]
    r = exp_return(data, P)
    vol = volatility(data, P)
    serr = vol / (len(data["Price in 1 Year"]) ** (1/2))
    interval[0] = r - 1.96 * serr
    interval[1] = r + 1.96 * serr
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    return interval

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [6]:
def exp_return_capm(beta, market_return, rf):
    capm = rf + (beta * (market_return - rf))
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    return capm

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [7]:
def observations(df):
    if len(df.index) < 500:
        return df.iloc[0]["CUSIP"]
    return np.NaN

In [8]:
def clean_data(path):
    df = pd.read_csv(path)
    mask = pd.to_numeric(df["RET"], errors="coerce").notnull()
    df = df[mask]
    df["RET"] = df["RET"].apply(float)
    grouped = df.groupby(by=["CUSIP"]).apply(observations).dropna()
    grouped = grouped.squeeze()
    grouped = grouped.tolist()
    df = df.set_index(df["CUSIP"])
    df = df.drop(grouped)
    df = df.reset_index(drop=True)
    # This should yield data with 403316 rows × 4 columns
    return df


Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [9]:
def ret(df):
    return sum(df["RET"]) / len(df["RET"])

In [10]:
def avg_return(data):
    # This should yield data with 679 rows × 2 columns
    grouped = data.groupby(by=["CUSIP"]).mean()
    
    output = pd.DataFrame(data = {
        "CUSIP" : grouped.index,
        "Average Return" : grouped["RET"].tolist()
    })
    
    return output

In [11]:
def vol(df):
    avg = sum(df["RET"]) / len(df.index)
    v = 0
    for x in range(len(df.index)):
        v += (df.iloc[x]["RET"] - avg) ** 2
    return (v  / (len(df.index) - 1)) ** (1/2)
    

In [12]:
def security_volatilities(data):
    # This should yield data with 679 rows × 2 columns
    
    grouped = data.groupby(by=["CUSIP"]).apply(vol)
    
    output = pd.DataFrame(data = {
        "CUSIP" : grouped.index.get_level_values(0).tolist(),
        "Volatility" : grouped.tolist()
    })
    
    return output

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [13]:
def correlation(ret_data, vol_data):
    # correlation(rets, vols) should yield 0.44837090727059614
    df = pd.merge(ret_data, vol_data, how='inner', on="CUSIP")
    df = df[["Average Return", "Volatility"]]
    
    return df.corr().iloc[0, 1]